# TEXT CLASSIFICATION / TOPIC MODELLING

## IMPORTING LIBRARIES

In [2]:
import re
import sys
import string
import random
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
#from sklearn.decomposition import TruncatedSVDs
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
#from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import ExtraTreesClassifier
#from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
#%matplotlib inline

C:\Users\Waqas\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## LOADING DATA

In [3]:
data = pd.read_excel('Training Data for Primary.xlsx',sheetname = 'Sheet1')
data = data.sample(frac=1) #shuffle data
data_pred = pd.read_excel('Primary to be decided.xlsx',sheetname = 'Sheet1')

LOOKING AT DATA **NON-STRATIFICATION**

In [470]:
# total_unique_values = pd.DataFrame(data['Existing PIC'].value_counts())
# total_unique_values

,Existing PIC
Application Software,31544
Movies and Entertainment,4565
Systems Software,4388
Interactive Media and Services,3179
Integrated Telecommunication Services,2410
Data Processing and Outsourced Services,1918
Internet Services and Infrastructure,1732
Financial Exchanges and Data,818
Internet and Direct Marketing Retail,108


DATA AFTER DROPPING SPECIFIC NO OF SAMPLES IN SPECIFIC CLASS.<br>DONE TO MAKE NO OF SENTENCES VARY IN SPECIFIC CLASS.<br>HELPS IN MANUAL ERROR ANALYSIS AT END BY COMPARING F1 SCORES. 

In [4]:
# ##decreasing data by removing 26000 APPLICATION SOFTWARE CLASS data points.
# app_index = data[data['Existing PIC'] == 'Application Software'].index.tolist()
# new_app_index = sample(app_index,26000)
# data.drop(new_app_index,inplace=True)

## DATA PRE-PROCESSING

REMOVING **STOP WORDS**, **PUNCTUATIONS**, **WHITESPACES** AND DOING **STEMMNG**, **LOWERING**

In [5]:
punc = string.punctuation
stemmer = SnowballStemmer('english')
words = stopwords.words("english")
punc = punc.replace("&", "") # don't remove &
pattern = r"[{}]".format(punc) # create the pattern

data['cleaned'] = data['Business Description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub(pattern, " ", x).split() if i not in words]).lower())
#data['cleaned'] = data['cleaned'].apply(lambda x: ''.join(ch for ch in x  if ch not in pattern))
data['cleaned'] = data['cleaned'].str.strip()

data_pred['cleaned'] = data_pred['Business Description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub(pattern, " ", x).split() if i not in words]).lower())
#data['cleaned'] = data['cleaned'].apply(lambda x: ''.join(ch for ch in x  if ch not in pattern))
data_pred['cleaned'] = data_pred['cleaned'].str.strip()

REMOVING SENTENCES HAVING LENGTH > 50.<br>THIS REMOVAL DOES NOT MAKE MUCH DIFFERENCE BECAUSE ONLY ABOUT 100 SAMPLES ARE GREATER THAN LENGTH 50 AND MOSTLY ARE FROM APPLICATIONS SOFTWARE<br><font color=red>**IT IS DONE FOR FUTURE PADDING PURPOSES IN BIDIRECTIONAL LSTM CELL**</font>

In [300]:
# data['len'] = data['cleaned'].str.split()
# data['len'] = data['len'].apply(lambda x: len(x))
# large_sent = data[data['len']>50]
# filter_data = data[data['len']<50]

# short_sentences = filter_data['cleaned']
# short_y = filter_data['Existing PIC']

CREATING DATA INPUTS AND OUTPUTS AFTER PRE-PROCESSING

In [8]:
X = data['cleaned']
y = data['Existing PIC']

X_pred = data_pred['cleaned']

# Data understanding

#### common words between application software class<font color=red>(having 62% of data)</font> and all other classes

In [477]:
# app_soft = data[data['Existing PIC']== 'Application Software']
# app_soft_counts = pd.Series(" ".join(app_soft['cleaned']).split()).value_counts()[:100]
# int_med = data[data['Existing PIC']== 'Interactive Media and Services']
# int_med_counts = pd.Series(" ".join(int_med['cleaned']).split()).value_counts()[:100]
# sys_soft = data[data['Existing PIC']== 'Systems Software']
# sys_soft_counts = pd.Series(" ".join(sys_soft['cleaned']).split()).value_counts()[:100]
# mov_ent = data[data['Existing PIC']== 'Movies and Entertainment']
# mov_ent_counts = pd.Series(" ".join(mov_ent['cleaned']).split()).value_counts()[:100]
# data_pro = data[data['Existing PIC']== 'Data Processing and Outsourced Services']
# data_pro_counts = pd.Series(" ".join(data_pro['cleaned']).split()).value_counts()[:100]
# int_ser = data[data['Existing PIC']== 'Internet Services and Infrastructure']
# int_ser_counts = pd.Series(" ".join(int_ser['cleaned']).split()).value_counts()[:100]
# int_tel = data[data['Existing PIC']== 'Integrated Telecommunication Services']
# int_tel_counts = pd.Series(" ".join(int_tel['cleaned']).split()).value_counts()[:100]
# fin_exc = data[data['Existing PIC']== 'Financial Exchanges and Data']
# fin_exc_counts = pd.Series(" ".join(fin_exc['cleaned']).split()).value_counts()[:100]
# int_dir = data[data['Existing PIC']== 'Internet and Direct Marketing Retail']
# int_dir_counts = pd.Series(" ".join(int_dir['cleaned']).split()).value_counts()[:100]

# common_int = pd.DataFrame({'int':int_med_counts,'app':app_soft_counts})
# common_sys = pd.DataFrame({'sys':sys_soft_counts,'app':app_soft_counts})
# common_mov = pd.DataFrame({'mov':mov_ent_counts,'app':app_soft_counts})
# common_data = pd.DataFrame({'data':data_pro_counts,'app':app_soft_counts})
# common_ser = pd.DataFrame({'ser':int_ser_counts,'app':app_soft_counts})
# common_tel = pd.DataFrame({'tel':int_tel_counts,'app':app_soft_counts})
# common_fin = pd.DataFrame({'fin':fin_exc_counts,'app':app_soft_counts})
# common_dir = pd.DataFrame({'dir':int_dir_counts,'app':app_soft_counts})

# Systems_Soft = common_sys.dropna(thresh=2).shape[0]
# Interactive_Media = common_int.dropna(thresh=2).shape[0]
# Movies_and_Enter = common_mov.dropna(thresh=2).shape[0]
# Data_Processing_and_Outsourced = common_data.dropna(thresh=2).shape[0]
# Internet_Services = common_ser.dropna(thresh=2).shape[0]
# Integrated_Telecommunication = common_tel.dropna(thresh=2).shape[0]
# Financial_Exchanges = common_fin.dropna(thresh=2).shape[0]
# Internet_and_Direct = common_dir.dropna(thresh=2).shape[0]

#### out of 100 most frequent words in all classes following are common no of words between application software class and specified class

In [488]:
# common_values = {'Systems_Software':Systems_Soft,
#                 'Interactive_Media_and_Services':Interactive_Media,
#                 'Movies_and_Entertainment':Movies_and_Enter,
#                 'Data_Processing_and_Outsourced_Services':Data_Processing_and_Outsourced,
#                 'Internet_Services_and_Infrastructure':Internet_Services,
#                 'Integrated_Telecommunication_Services':Integrated_Telecommunication,
#                 'Financial_Exchanges_and_Data':Financial_Exchanges,
#                 'Internet_and_Direct_Marketing_Retail':Internet_and_Direct}
common_values

{'Data_Processing_and_Outsourced_Services': 69,
 'Financial_Exchanges_and_Data': 51,
 'Integrated_Telecommunication_Services': 49,
 'Interactive_Media_and_Services': 63,
 'Internet_Services_and_Infrastructure': 68,
 'Internet_and_Direct_Marketing_Retail': 45,
 'Movies_and_Entertainment': 41,
 'Systems_Software': 75}

# FEATURE EXTRACTION

#### PRE-TRAINED WORD EMBEDDINGS

TWO METHODS ARE USED TO EXTRACT SENTENCE VECTOR OUT OF WORD VECTORS:<br>
<font color=red>**1. TAKING MEAN OF ALL VECTORS.**<br>
**2. TAKING MAX,MIN OF ALL VECTORS AND CONCATENATING THEM**</font>

In [348]:
# class GloveVectorizer:
#     def __init__(self):# load in pre-trained word vectors
#         print('Loading word vectors...')
#         word2vec = {}
#         embedding = []
#         idx2word = []
#         with open('glove.6B.300d.txt',encoding= 'utf-8') as f:
#             # is just a space-separated text file in the format:
#           # word vec[0] vec[1] vec[2] ...
#           for line in f:
#                 values = line.split()
#                 word = values[0]
#                 vec = np.asarray(values[1:], dtype='float32')
#                 word2vec[word] = vec
#                 embedding.append(vec)
#                 idx2word.append(word)
#                 #print('Found %s word vectors.' % len(word2vec)) for visualization

#                 # save for later
#                 self.word2vec = word2vec
#                 self.embedding = np.array(embedding)
#                 self.word2idx = {v:k for k,v in enumerate(idx2word)}
#                 self.V, self.D = self.embedding.shape

#     def fit(self, data):
#         pass

#     def transform(self, data):
#         X = np.zeros((len(data), self.D))
#         n = 0
#         emptycount = 0
#         for sentence in data:
#             tokens = sentence.lower().split()
#             vecs = []
#             for word in tokens:
#                 if word in self.word2vec:
#                     vec = self.word2vec[word]
#                     vecs.append(vec)
#             if len(vecs) > 0:
#                 vecs = np.array(vecs)
#                 max_vec = vecs.max(axis=0)
#                 #min_vec = vecs.min(axis=0)
#                 #X[n] = vecs.mean(axis=0)
#                 #X[n] = np.concatenate((max_vec,min_vec))
#                 X[n] = max_vec
#             else:
#                 emptycount += 1
#             n += 1
#         print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
#         return X

#     def fit_transform(self, data):
#         self.fit(data)
#         return self.transform(data)

# glove_vectorizer = GloveVectorizer()

In [349]:
# class Word2VecVectorizer:
#     def __init__(self):
#         print("Loading in word vectors...")
#         self.word_vectors = KeyedVectors.load_word2vec_format(
#         'GoogleNews-vectors-negative300.bin',binary=True
#             )
#         print("Finished loading in word vectors")
        
#     def fit(self, data):
#         pass

#     def transform(self, data):
#         # determine the dimensionality of vectors
#         v = self.word_vectors.get_vector('king')
#         self.D = v.shape[0]

#         X = np.zeros((len(data), self.D))
#         n = 0
#         emptycount = 0
#         for sentence in data:
#             tokens = sentence.split()
#             vecs = []
#             m = 0
#             for word in tokens:
#                 try:
#                     # throws KeyError if word not found
#                     vec = self.word_vectors.get_vector(word)
#                     vecs.append(vec)
#                     m += 1
#                 except KeyError:
#                     pass
#             if len(vecs) > 0:
#                 vecs = np.array(vecs)
#                 max_vec = vecs.max(axis=0)
#                 #min_vec = vecs.min(axis=0)
#                 #X[n] = vecs.mean(axis=0)
#                 #X[n] = np.concatenate((max_vec,min_vec))
#                 X[n] = max_vec
#             else:
#                 emptycount += 1
#             n += 1
#         print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
#         return X


#       def fit_transform(self, data):
#             self.fit(data)
#             return self.transform(data)

# word_vectorizer = Word2VecVectorizer()

#### USING GLOVE/WORD2VEC AFTER LOADING

In [350]:
#X = glove_vectorizer.fit_transform(X)
#X = word_vectorizer.fit_transform(X)

#### COUNTER / TF-IDF VECTORIZERS USING N-GRAMS

In [9]:
# ### COUNTER
vect = CountVectorizer(ngram_range=(1, 3), stop_words="english")
vect.fit(X)
X = vect.transform(X)

X_pred = vect.transform(X_pred)

# ### TFIDF
# #vect = TfidfVectorizer(ngram_range=(1, 1))
# #vect.fit(X)
# #X = vect.transform(X)

#### USING CHI DISTRIBUTION/SVD ETC FOR TOP 20000 FEATURES SELECTION

In [10]:
chi = SelectKBest(chi2,20000)
chi.fit_transform(X,y)

#svd = TruncatedSVD(20000)
#X = svd.fit_transform(X)

<50662x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 725760 stored elements in Compressed Sparse Row format>

#  HANDLING NON-STRATIFICATION

In [353]:
#sm = SMOTE()
#X, y = SMOTE(kind = 'svm').fit_sample(X,y)
#class_weights = {'Application Software':0.1,'Movies and Entertainment':1,'Systems Software':10,
#                 'Interactive Media and Services':1,'Integrated Telecommunication Services':10,
#                'Data Processing and Outsourced Services':1,'Internet Services and Infrastructure':1,
#                'Financial Exchanges and Data':1,'Internet and Direct Marketing Retail':3}
#THESE CLASS WEIGHTS ARE USED IN LINEARSVC

# MAKING MODEL

In [11]:
clf = LinearSVC(penalty='l1',dual=False)            
model = clf.fit(X, y)

# MAKING PREDICTION

In [12]:
predicted_train = model.predict(X)
data_pred['Proposed PIC'] = model.predict(X_pred)

In [15]:
f1_score(predicted_train,y,average = 'weighted')

0.9999802603718465

In [19]:
data_pred.drop('cleaned',axis = 1,inplace = True)
data_pred.head()

,Business Description,CIQ Company ID,Proposed PIC
0,"The Burgiss Group, LLC develops and offers inv...",19351,Financial Exchanges and Data
1,"Lycos, Inc. is an Internet company that operat...",21563,Interactive Media and Services
2,Providers of online services for IT profession...,21871,Interactive Media and Services
3,"SkyDesk, Inc. provides online data storage and...",24078,Internet Services and Infrastructure
4,The company provides an Internet based browse...,24159,Application Software


writing to excel file

In [377]:
writer = pd.ExcelWriter('Predicted Results.xlsx')
data_pred.to_excel(writer,'Sheet1')
writer.save()

## CREATING CONFUSION MATRIX

CONFUSION MATRIX IS BEST FOR MANUAL ERROR ANALYSIS.<br>
LOOKING AT CONFUSION MATRIX FURTHER HYPER-HYPER PARAMETERS COULD BE TUNED

In [ ]:
# con_mat = pd.DataFrame(confusion_matrix(predicted_test,y_test),index = unique_values.index,columns = unique_values.index)
# con_mat

## kappa, f1_score for non-stratified data

In [309]:
# #train f1_score
# f1_Score = metrics.f1_score(predicted_train,y_train,average = 'weighted')
# f1_Score

In [310]:
# #test f1_score
# f1_Score = metrics.f1_score(predicted_test,y_test,average = 'weighted')
# f1_Score

In [311]:
# # kappa train accuracy
# metrics.cohen_kappa_score(predicted_train,y_train)

In [312]:
# # kappa test accuracy
# metrics.cohen_kappa_score(predicted_test,y_test)

## misclassified classes and their samples

In [313]:
# misclassified_data = y_test[predicted_test != y_test].value_counts()
# misclassify_data = pd.DataFrame(misclassified_data)
# misclassify_data.columns = ['misclassified no of samples']
# misclassify_data

## accuracy per class

In [314]:
# perc_acc = pd.DataFrame(1-(misclassify_data['misclassified no of samples']/unique_values['test_unique_values']))
# perc_acc.columns = ['percentage accuracy per class']
# perc_acc